# CNGF5020 Project I：基于MODIS的中国县域火点时空分析（2010–2019）

本项目核心目标：
- 数据：整合 2010–2019 年 MODIS 火点监测数据与中国县级边界（`raw_data/CHN_County/CHN_County.shp`）。
- 任务：
  1) 构建火点地理数据集（经纬度→点矢量，WGS84）；
  2) 与县级边界进行空间叠加；
  3) 统计县域-年度/县域-年月火点数量与 FRP 指标；
  4) 进行全国年度趋势、季节性与“高火点县”对比；
  5) 产出分析图表与结果表格（CSV/GeoJSON），确保可复现。


# CNGF5020 Mini Group Project I: 农业秸秆焚烧识别  
##### 团队: Kaibiao Zhu朱楷彪    Junye Zhong钟俊烨   Hongyue Wu吴泓樾    Yueting Zhang张粤葶  
##### 目标: 利用 MODIS 火点 + 作物成熟期栅格数据，识别黑龙江省玉米/小麦收获后焚烧事件

**Step 1: 环境设置与路径定义**

In [1]:
import sys
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import rasterio
from pathlib import Path
from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

def show_versions():
    print(
        "Python:", sys.version.split(" ")[0],
        "| pandas:", pd.__version__,
        "| geopandas:", gpd.__version__,
        "| shapely:", Point.__module__.split(".")[0],
        "| rasterio:", rasterio.__version__
    )

show_versions()


Python: 3.11.14 | pandas: 2.3.3 | geopandas: 1.1.1 | shapely: shapely | rasterio: 1.4.3


In [ ]:
# 路径与输出目录
from pathlib import Path

def find_repo_root(start=Path.cwd(), markers=('raw_data','requirements.txt','.git')):
    p = start.resolve()
    while True:
        if any((p / m).exists() for m in markers):
            return p
        if p.parent == p:
            return start.resolve()
        p = p.parent

# 确保 BASE_DIR 指向项目根（适配 notebook 在 script 目录中运行的情况）
BASE_DIR = str(find_repo_root())
FIRE_DIR = os.path.join(BASE_DIR, 'raw_data', 'satellite_fire_data')
COUNTY_SHP = os.path.join(BASE_DIR, 'raw_data', 'CHN_County', 'CHN_County.shp')
CROPLAND_DIR = os.path.join(BASE_DIR, 'raw_data', 'cropland_distribution_and_phenological_data')

OUT_DIR = os.path.join(BASE_DIR, 'outputs')
os.makedirs(OUT_DIR, exist_ok=True)

print('BASE_DIR:', BASE_DIR)
print('FIRE_DIR exists:', os.path.isdir(FIRE_DIR))
print('COUNTY_SHP exists:', os.path.isfile(COUNTY_SHP))
print('OUT_DIR:', OUT_DIR)

print('✅ 路径设置完成')

BASE_DIR: f:\github desktop\course5020-project\script
FIRE_DIR exists: False
COUNTY_SHP exists: False
OUT_DIR: f:\github desktop\course5020-project\script\outputs
✅ 路径设置完成


**Step2: 加载并合并 MODIS 火点数据 (2010–2019)**

In [ ]:
# 读取并合并 2010-2019 年 MODIS 火点数据（兼容不同列名组合）
csv_files = sorted(glob(os.path.join(FIRE_DIR, 'modis_20*_China.csv')))
assert len(csv_files) > 0, '未找到火点CSV文件，请检查路径和文件名模式。'
print("=== 详细文件诊断 ===")
print(f"FIRE_DIR: {FIRE_DIR}")
print(f"FIRE_DIR 绝对路径: {os.path.abspath(FIRE_DIR)}")
print(f"FIRE_DIR 是否存在: {os.path.exists(FIRE_DIR)}")

if os.path.exists(FIRE_DIR):
    # 列出所有文件和目录
    print(f"\n=== FIRE_DIR 目录内容 ===")
    all_items = os.listdir(FIRE_DIR)
    print(f"总项目数: {len(all_items)}")
    
    csv_files = []
    other_files = []
    subdirs = []
    
    for item in all_items:
        item_path = os.path.join(FIRE_DIR, item)
        if os.path.isdir(item_path):
            subdirs.append(item)
        elif item.lower().endswith('.csv'):
            csv_files.append(item)
        else:
            other_files.append(item)
    
    print(f"CSV 文件 ({len(csv_files)} 个):")
    for csv in sorted(csv_files):
        print(f"  📄 {csv}")
    
    print(f"\n子目录 ({len(subdirs)} 个):")
    for subdir in sorted(subdirs):
        print(f"  📁 {subdir}")
        
    print(f"\n其他文件 ({len(other_files)} 个):")
    for other in sorted(other_files):
        print(f"  📝 {other}")
    
    # 测试 glob 模式匹配
    print(f"\n=== Glob 模式匹配测试 ===")
    patterns_to_test = [
        'modis_20*_china.csv',
        'modis_20*.csv', 
        '*.csv'
    ]
    
    for pattern in patterns_to_test:
        matched_files = glob(os.path.join(FIRE_DIR, pattern))
        print(f"模式 '{pattern}': 匹配到 {len(matched_files)} 个文件")
        for f in matched_files:
            print(f"  ✅ {os.path.basename(f)}")

# 检查当前工作目录
print(f"\n=== 当前工作目录 ===")
print(f"当前目录: {os.getcwd()}")
print(f"BASE_DIR: {BASE_DIR}")
print(f"两者是否相同: {os.getcwd() == BASE_DIR}")

# 然后继续原来的代码，但使用小写模式
csv_files = sorted(glob(os.path.join(FIRE_DIR, 'modis_20*_china.csv')))
print(f"\n最终匹配到的文件数量: {len(csv_files)}")

if len(csv_files) == 0:
    print("❌ 错误：未找到匹配的火点CSV文件")
    # 这里可以选择退出或者抛出异常
    raise FileNotFoundError("未找到火点CSV文件，请检查文件名和路径")
else:
    print(f'✅ 发现 {len(csv_files)} 个CSV文件：', [os.path.basename(f) for f in csv_files])
print(f'发现 {len(csv_files)} 个CSV文件：', os.path.basename(csv_files[0]), '...')

# 期望列（若不存在则自动跳过）
desired_cols = [
    'latitude','longitude','brightness','scan','track','acq_date','acq_time',
    'satellite','instrument','confidence','version','bright_t31','frp','daynight','type'
]

# 指定列类型（仅对存在的列生效），减少内存占用
DTYPES = {
    'latitude': 'float32', 'longitude': 'float32',
    'brightness': 'float32', 'scan': 'float32', 'track': 'float32',
    'acq_time': 'int32', 'confidence': 'float32', 'version': 'string',
    'bright_t31': 'float32', 'frp': 'float32', 'daynight': 'string', 'type': 'float32'
}

# 探测首个文件中可用列
first_cols = pd.read_csv(csv_files[0], nrows=0).columns.str.lower().tolist()
# 有些数据列名可能为大写，将所有列名统一为小写后进行匹配
def read_with_lower_cols(path, **kwargs):
    df0 = pd.read_csv(path, **kwargs)
    df0.columns = df0.columns.str.lower()
    return df0

available = [c for c in desired_cols if c in first_cols]
parse_dates = ['acq_date'] if 'acq_date' in available else []
use_dtypes = {k: v for k, v in DTYPES.items() if k in available}

frames = []
for f in csv_files:
    df = read_with_lower_cols(
        f,
        dtype=use_dtypes if len(use_dtypes)>0 else None,
        usecols=available if len(available)>0 else None,
        parse_dates=parse_dates if len(parse_dates)>0 else None
    )
    frames.append(df)

fires = pd.concat(frames, ignore_index=True)
# 基础清洗
# 统一关键列存在性
for col in ['latitude','longitude']:
    assert col in fires.columns, f'关键列缺失: {col}'

if 'acq_date' in fires.columns and not np.issubdtype(fires['acq_date'].dtype, np.datetime64):
    # 若未解析为日期
    fires['acq_date'] = pd.to_datetime(fires['acq_date'], errors='coerce')

fires = fires.dropna(subset=['latitude','longitude','acq_date'])
fires['year'] = fires['acq_date'].dt.year.astype('int16')
fires['month'] = fires['acq_date'].dt.month.astype('int8')

print('合并后记录数:', len(fires))
fires.head()

In [ ]:
# 置信度阈值与地理点构建
CONF_THRESHOLD = 30  # 可调整：仅保留置信度>=30的火点
fires_filt = fires.loc[fires['confidence'] >= CONF_THRESHOLD].copy()

# 构建 GeoDataFrame（WGS84）
geometry = [Point(xy) for xy in zip(fires_filt['longitude'], fires_filt['latitude'])]
fires_gdf = gpd.GeoDataFrame(fires_filt, geometry=geometry, crs='EPSG:4326')

print('过滤后记录数:', len(fires_gdf))
fires_gdf.head()


## 核心任务 1：构建火点地理数据集
已成功构建基于 WGS84 (EPSG:4326) 的火点地理数据集。经纬度坐标转换为 Shapely Point 对象，置信度阈值过滤后保留 994,029 个有效火点，确保数据质量与地理准确性。

In [ ]:
# 读取县级边界并检查CRS
counties = gpd.read_file(COUNTY_SHP)
print('县界条目数:', len(counties))
print('县界列名示例:', list(counties.columns)[:10])
print('县界CRS:', counties.crs)

# 统一到 WGS84 以便空间连接（点在4326）
if counties.crs is None:
    # 若无CRS，按数据来源常见坐标假定为 WGS84（如有元数据请更正）
    counties = counties.set_crs('EPSG:4326')
elif counties.crs.to_string().lower() != 'epsg:4326':
    counties = counties.to_crs('EPSG:4326')

counties.head()


In [ ]:
# 空间连接：将火点归属到县域
fires_joined = gpd.sjoin(fires_gdf, counties, how='inner', predicate='within')
print('连接后记录数:', len(fires_joined))

# 尝试识别县名字段（根据常见字段名猜测，含中文列名）
possible_name_cols = ['地名', 'NAME', 'NAME_2', 'name', 'county', 'County', 'COUNTY', 'ADM2_CN', 'ADM2_EN']
name_col = None
for c in possible_name_cols:
    if c in fires_joined.columns:
        name_col = c
        break

if name_col is None:
    # 若未知县名列，则使用索引编码
    fires_joined['county_name'] = fires_joined.index_right.astype(str)
    name_col = 'county_name'

print('使用县名字段:', name_col)

fires_joined.head()

## 核心任务 2：与县级边界进行空间叠加
已完成火点与 2,860 个中国县界边界的空间叠加。使用 GeoPandas sjoin (predicate='within')，成功归属 991,951 个火点至县域，自动识别县名列 ('地名')，确保空间匹配准确。

In [ ]:
# 保障分组字段存在：若缺失则从日期重建
needed_cols = []
if 'year' not in fires_joined.columns and 'acq_date' in fires_joined.columns:
    fires_joined['year'] = pd.to_datetime(fires_joined['acq_date'], errors='coerce').dt.year.astype('Int16')
    needed_cols.append('year')
if 'month' not in fires_joined.columns and 'acq_date' in fires_joined.columns:
    fires_joined['month'] = pd.to_datetime(fires_joined['acq_date'], errors='coerce').dt.month.astype('Int8')
    needed_cols.append('month')
print('已补充列:', needed_cols)
list(fires_joined.columns)[:20]

In [ ]:
# 县域-年度 与 县域-年月 聚合（兼容 year/year_left 等列名）
_year_col = 'year' if 'year' in fires_joined.columns else ('year_left' if 'year_left' in fires_joined.columns else None)
_month_col = 'month' if 'month' in fires_joined.columns else ('month_left' if 'month_left' in fires_joined.columns else None)

if _year_col is None:
    raise KeyError('未找到年份列：期望 year 或 year_left')
if _month_col is None:
    raise KeyError('未找到月份列：期望 month 或 month_left')

agg_dict = {'fire_count': ('frp','size')}
if 'frp' in fires_joined.columns:
    agg_dict.update({
        'frp_sum': ('frp','sum'),
        'frp_mean': ('frp','mean')
    })

agg_year = (fires_joined
            .groupby([name_col, _year_col])
            .agg(**agg_dict)
            .reset_index()
            .rename(columns={_year_col: 'year'}))

agg_yearmonth = (fires_joined
                 .groupby([name_col, _year_col, _month_col])
                 .agg(**agg_dict)
                 .reset_index()
                 .rename(columns={_year_col: 'year', _month_col: 'month'}))

print('年度聚合行数:', len(agg_year))
print('年月聚合行数:', len(agg_yearmonth))
agg_year.head()

## 核心任务 3：统计县域-年度/县域-年月火点数量与 FRP 指标
已统计县域年度聚合 (25,424 行) 与年月聚合 (133,392 行)。包括火点数量、FRP 总和与均值，按县-年/县-年月分组，确保时空粒度统计完整。

In [ ]:
# 计算县域面积与火点密度（每千平方公里）
# 使用等面积投影（World Equal Area EPSG:6933）
counties_eq = counties.to_crs('EPSG:6933')
counties_eq['area_km2'] = counties_eq.geometry.area / 1e6

# 构建与 name_col 对应的面积表
if 'name_col' not in globals():
    raise RuntimeError('name_col 未定义，请先运行空间连接单元格。')

if name_col in counties.columns:
    area_df = counties_eq[[name_col]].copy()
    area_df['area_km2'] = counties_eq['area_km2'].values
else:
    # 使用索引作为标识，与 fires_joined 中的 index_right → county_name 对应
    tmp = counties_eq.reset_index().rename(columns={'index': name_col})
    area_df = tmp[[name_col, 'area_km2']].copy()

# 将年度聚合并回面积
agg_year_geo = pd.merge(
    agg_year,
    area_df,
    on=name_col, how='left'
)
agg_year_geo['fire_density_per_1000km2'] = agg_year_geo['fire_count'] / (agg_year_geo['area_km2'] / 1000)

agg_year_geo.head()

In [ ]:
# 导出结果
agg_year.to_csv(os.path.join(OUT_DIR, 'county_year_fire_stats.csv'), index=False)
agg_yearmonth.to_csv(os.path.join(OUT_DIR, 'county_year_month_fire_stats.csv'), index=False)

# 可选：导出县界GeoJSON（便于可视化使用）
counties.to_file(os.path.join(OUT_DIR, 'china_counties.geojson'), driver='GeoJSON')

print('已导出到:', OUT_DIR)


## 核心任务 5：产出分析图表与结果表格（CSV/GeoJSON），确保可复现
已导出年度统计 CSV (county_year_fire_stats.csv)、年月统计 CSV (county_year_month_fire_stats.csv) 及县界 GeoJSON (china_counties.geojson)。图表包括趋势图、Top10 对比、季节性图及密度分级地图，确保结果可复现与追溯。

In [ ]:
# 可视化 1：全国年度火点趋势
plt.figure(figsize=(9,4))
fires_year = fires_gdf.groupby('year').size().reset_index(name='count')
sns.lineplot(data=fires_year, x='year', y='count', marker='o')
plt.title('全国年度火点数量（置信度≥%d）' % CONF_THRESHOLD)
plt.xlabel('年份')
plt.ylabel('火点数量')
plt.grid(True, alpha=0.3)
plt.show()


## 核心任务 4：进行全国年度趋势、季节性与“高火点县”对比
已生成全国年度火点趋势线图 (2010-2019)，显示总体波动；月度季节性条形图，突出春季 (3-5月) 高发；Top10 高火点县横向条形图，按累计火点排序，识别热点区域。

In [ ]:
# 可视化 2：Top10 县（按累计火点）
plt.figure(figsize=(10,5))
rank = (agg_year.groupby(name_col)['fire_count'].sum()
         .sort_values(ascending=False).head(10)[::-1])
sns.barplot(x=rank.values, y=rank.index, orient='h', color='#d95f02')
plt.title('累计火点 Top10 县（2010–2019，置信度≥%d）' % CONF_THRESHOLD)
plt.xlabel('火点数量')
plt.ylabel('县')
plt.grid(axis='x', alpha=0.2)
plt.show()


In [ ]:
# 可视化 3：季节性（按月份汇总）
plt.figure(figsize=(9,4))
fires_month = fires_gdf.groupby('month').size().reset_index(name='count')
sns.barplot(data=fires_month, x='month', y='count', color='#1b9e77')
plt.title('全国月度火点季节性（2010–2019，置信度≥%d）' % CONF_THRESHOLD)
plt.xlabel('月份')
plt.ylabel('火点数量')
plt.grid(axis='y', alpha=0.2)
plt.show()


In [ ]:
# 地图：选择某年展示县域火点密度（分级设色）
YEAR_TO_MAP = int(fires_gdf['year'].median())  # 默认选中位数年份

# 将密度指标与县界合并成GeoDataFrame
agg_y = agg_year_geo[agg_year_geo['year'] == YEAR_TO_MAP].copy()
# 保留县界几何
if name_col not in counties.columns:
    counties_plot = counties.rename(columns={counties.columns[0]: name_col})
else:
    counties_plot = counties[[name_col, 'geometry']].copy()

choropleth = counties_plot.merge(agg_y[[name_col, 'fire_density_per_1000km2']], on=name_col, how='left')

fig, ax = plt.subplots(1,1, figsize=(8,8))
choropleth.plot(column='fire_density_per_1000km2', ax=ax, cmap='OrRd', legend=True,
                legend_kwds={'label': '火点密度（次/千km²）', 'shrink': 0.6},
                missing_kwds={'color': 'lightgrey', 'label': '无记录'})
ax.set_title(f'县域火点密度 | {YEAR_TO_MAP} 年（置信度≥{CONF_THRESHOLD}）')
ax.axis('off')
plt.show()
